# Предмет исследования: **мобильная игра Quiz Freez**

Необходимо проверить 3 гипотезы:

1. Влияет ли обучение на скорость прохождения других этапов игры?
2. Повторное прохождение обучения положительно влияет на оплату, верно?
3. Если пользователь сначала выбирает сложность обучения, будет ли он потом проходить обучение?

Основные этапы пути пользователя:


Регистрация (registration) — это обязательный этап. Без регистрации
пользователь не может пройти на следующие этапы работы с приложением.

Старт обучения (tutorial start) — опциональный этап. Пользователь после регистрации может перейти к обучению работе с приложением, а может и не перейти. При этом вернуться к обучению можно в любой момент. А ещё можно пройти обучение несколько раз.

Завершение обучения (tutorial finish) может произойти только в случае, если ранее произошло событие «Старт обучения», но при этом пользователь может не завершить обучение.


Выбор уровня сложности вопросов (level choice) — это обязательное событие, которое нужно для того, чтобы перейти к выбору пакетов вопросов. Таким образом, пользователь может не пройти обучение или даже не начинать его, но прежде чем начать отвечать, он обязан выбрать уровень сложности.

Выбор пакетов вопросов (pack choice, другое название training choice) — это этап, на котором пользователь выбирает себе бесплатный набор пакетов вопросов, на которые он будет отвечать.

Покупка платных пакетов вопросов (purchase) — это факт совершения оплаты за вопросы, которые не доступны в списке бесплатных вопросов.

# ЦЕЛЬ
Исследовать поведение пользователей в обновлённом приложении, которые зарегистрировались с 1 января по 31 декабря 2018 года включительно.

# ЗАДАЧИ
В рамках проверки гипотез вам необходимо решить три задачи:

1. Определить, насколько обучение сокращает время прохождения этапов игры.
2. Доказать, что успешное обучение само по себе влияет на оплату и не имеет значения то, каким этапом оно шло.
3. Определить, насколько прозрачен процесс взаимодействия с игрой.




# КОНКРЕТНЫЕ ШАГИ (ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ)
1. Сравнить время прохождения различных этапов для пользователей, которые завершили обучение, и пользователей, не начинавших обучение. Если показатель отличается, выяснить, насколько.
2. Проверить, существует ли зависимость между вероятностью оплаты вопросов и количеством обучений, которые начинал или завершал пользователь. 
3. Выяснить, как часто пользователи начинают обучение после выбора уровня сложности. (Это позволит оценить прозрачность процесса взаимодействия с игрой: если пользователи после выбора уровня сложности обращаются к обучению, значит, работа с приложением непонятна.)

# Исходные данные

> Таблица **Event**

Хранит данные о событиях, которые совершают пользователи. По сути, каждое событие — это факт прохождения пользователем какого-либо этапа игры.

Название поля	Описание
**id** - идентификатор события

**user_id** -	уникальный идентификатор пользователя, совершившего событие в приложении

**start_time** - дата и время события

**event_type** -	тип события (значения: registration — регистрация; tutorial_start — начало обучения; tutorial_finish — завершение обучения; level_choice — выбор уровня сложности; pack_choice — выбор пакетов вопросов)

**tutorial_id** -	идентификатор обучения (этот идентификатор есть только у событий обучения)

**selected_level** - выбранный уровень сложности обучения

> Таблица **purchase**

Хранит данные об оплатах, которые совершают пользователи.

Название поля	Описание
**id** -	идентификатор события

**user_id** -	уникальный идентификатор пользователя, совершившего событие в приложении

**event_datetime** -	дата и время события/покупки

**amount** - сумма оплаты

# Проверка гипотезы № 1
Определить, насколько обучение сокращает время прохождения этапов игры.

In [1]:
import pandas as pd
import plotly.express as px

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Считываем данные
events_df = pd.read_csv('/content/drive/My Drive/GD-3.Project _1/7_4_Events.csv')
purchase_df = pd.read_csv('/content/drive/My Drive/GD-3.Project _1/purchase.csv')

In [ ]:
# Выделяем пользователей, которые зарегестрировались в 2018 году. Переводим время в обеих таблицах в формат datetime.
mask_1 = (events_df['event_type'] == 'registration') & (events_df['start_time'] >= '2018-01-01') & (events_df['start_time'] <= '2018-12-31')
registration_df_2018 = events_df[mask_1]['user_id']
events_df_2018 = events_df[events_df['user_id'].isin(registration_df_2018)]
purchase_df_2018 = purchase_df[purchase_df['user_id'].isin(registration_df_2018)]
events_df_2018['start_time'] = pd.to_datetime(events_df_2018['start_time'])
purchase_df_2018['event_datetime'] = pd.to_datetime(purchase_df_2018['event_datetime'])

In [ ]:
#Выделяем пользователей, которые прошли обучение events_df_2018
useres_complete_tutorial = events_df_2018[events_df_2018['event_type'] == 'tutorial_finish']['user_id']
events_df_2018_useres_complete_tutorial = events_df_2018[events_df_2018['user_id'].isin(useres_complete_tutorial)]

In [ ]:
#Переименовываем наименования колонок таблицы purchase_2018 и отбираем только тех пользователей, которые прошли обучение.
purchase_2018 = purchase_df.rename(columns={"id": "purchase_id","event_datetime": "start_time" })
purchase_2018['event_type'] = 'purchase'
purchase_2018_comp_tut = purchase_2018[purchase_2018['user_id'].isin(useres_complete_tutorial)]

In [ ]:
#Строим сводную таблицу user_id от event_type по start_time. Время берем минимальное по каждому пользователю.
events_df_2018_useres_complite_tutorial=events_df_2018_useres_complete_tutorial[['event_type', 'start_time', 'user_id' ]]
purchase_2018_comp_tut = purchase_2018_comp_tut[['event_type', 'start_time', 'user_id' ]]
total_table = pd.concat([events_df_2018_useres_complete_tutorial, purchase_2018_comp_tut])
pivot_total_table = total_table.groupby(['user_id', 'event_type'])['start_time'].min().unstack()
pivot_total_table


In [ ]:
#Находим разницу во времени между оплатой и регистрацией пользователей, прошедших обучение. 
#Находим среднее время между регистрацией и оплатой пользователей, прошедших обучение.
pivot_total_table['delta_time_pur_reg_for_tutorial'] = pivot_total_table['purchase'] - pivot_total_table['registration']
pivot_total_table = pivot_total_table.reindex(columns=['registration', 'tutorial_start', 'tutorial_finish', 'level_choice', 'pack_choice', 'purchase', 'delta_time_pur_reg_for_tutorial'])
pivot_total_table['delta_time_pur_reg_for_tutorial'].mean()
print(f"Среднее время между регистрацией и оплатой пользователей, прошедших обучение"
          f"  {pivot_total_table['delta_time_pur_reg_for_tutorial'].mean()}")

In [ ]:
#Находим пользователей, которые не начинали обучение или начали, но не закончили для обеих таблиц.
useres_without_tutorial = events_df_2018[events_df_2018['event_type'] != 'tutorial_finish']['user_id']
events_df_2018_useres_not_tutorial = events_df_2018[events_df_2018['user_id'].isin(useres_without_tutorial)]
purchase_2018_not_tut = purchase_2018[purchase_2018['user_id'].isin(useres_without_tutorial)]

In [ ]:
#Строим сводную таблицу user_id от event_type по start_time. Время берем минимальное по каждому пользователю.
events_df_2018_useres_not_tutorial=events_df_2018_useres_not_tutorial[['event_type', 'start_time', 'user_id' ]]
purchase_2018_not_tut = purchase_2018_not_tut[['event_type', 'start_time', 'user_id' ]]
total_table_not_tut = pd.concat([events_df_2018_useres_not_tutorial, purchase_2018_not_tut])
pivot_total_table_not_tut = total_table_not_tut.groupby(['user_id', 'event_type'])['start_time'].min().unstack()
pivot_total_table_not_tut

In [ ]:
#Находим разницу во времени между оплатой и регистрацией пользователей, не прошедших обучение. 
#Находим среднее время между регистрацией и оплатой пользователей, не прошедших обучение.
pivot_total_table_not_tut['delta_time_pur_reg_without_tutorial'] = pivot_total_table_not_tut['purchase'] - pivot_total_table_not_tut['registration']
pivot_total_table_not_tut = pivot_total_table_not_tut.reindex(columns=['registration', 'tutorial_start', 'tutorial_finish', 'level_choice', 'pack_choice', 'purchase', 'delta_time_pur_reg_without_tutorial'])
print(f"Среднее время между регистрацией и оплатой пользователей, не прошедших обучение"
          f"  {pivot_total_table_not_tut['delta_time_pur_reg_without_tutorial'].mean()}")

In [ ]:
#Строим столбчатую диаграмму по резуьльтатам проверки гипотезы № 1
avg_time_without_tut = pivot_total_table_not_tut['delta_time_pur_reg_without_tutorial'].mean()
avg_time_with_tut = pivot_total_table['delta_time_pur_reg_for_tutorial'].mean()

resultt = pd.Series(
    data = [str(avg_time_without_tut), str(avg_time_with_tut)],
    index = ["Среднее время между регистрацией и оплатой пользователей, не прошедших обучение", "Среднее время между регистрацией и оплатой пользователей, прошедших обучение" ]
)
avg_time_without_tut
fig__1 = px.bar(
    data_frame=resultt, #DataFrame
    x = resultt.index,
    y = [str(avg_time_with_tut),str(avg_time_without_tut),],
    text = [str(avg_time_without_tut), str(avg_time_with_tut)] , #текст на столбцах
    orientation='v', #ориентация графика
    height=500, #высота
    width=1000)
fig__1.show()


# Вывод по результатам проверки гипотезы № 1
Среднее время между регистрацией и оплатой пользователей, прошедших обучение  4 days 00:30:03.247408431

Среднее время между регистрацией и оплатой пользователей, не прошедших обучение  4 days 01:01:56.595000

Проверка гипотезы № 1 показала, что прохождение обучения не существенно влияет на время прохождения основных этапов пути пользователя. 

# Проверка гипотезы № 2 
Повторное прохождение обучения положительно влияет на оплату, верно?

In [ ]:
#Выделяем пользователей, которые закончили обучение или начинали
users_start_tutorial = events_df_2018[events_df_2018['event_type'] == 'tutorial_start']['user_id']
events_df_2018_users_start_tutorial = events_df_2018[events_df_2018['user_id'].isin(users_start_tutorial)]

In [ ]:
#Строим сводную таблицу, чтобы найти пользователей, которые несколько раз начинали обучение или несколько раз закончили
#Найдем кол-во пользователей
pivot_start_tut = events_df_2018_users_start_tutorial.groupby(['user_id', 'event_type'])['event_type'].count().unstack()
pivot_start_tut_more_1 = pivot_start_tut[(pivot_start_tut['tutorial_start']>=2) | (pivot_start_tut['tutorial_finish']>=2)]
users_start_tut_more_1 = pivot_start_tut_more_1.index
count_users_start_tut_more_1=len(users_start_tut_more_1)
count_users_start_tut_more_1

2755

In [ ]:
#Найдем кол-во пользователей, которые прошли обучение несколько раз и оплатили
purchase_df_2018_pur_start_tut = purchase_df_2018[purchase_df_2018['user_id'].isin(users_start_tut_more_1)]
count_users_start_tut_more_1_and_pur = purchase_df_2018_pur_start_tut['user_id'].nunique()
count_users_start_tut_more_1_and_pur

371

In [ ]:
#Найдем процент пользователей, которые прошли или начинали обучение несколько раз и оплатили
percent_user_more_tut_and_purch = round(count_users_start_tut_more_1_and_pur/count_users_start_tut_more_1 *100,2)
percent_user_more_tut_and_purch
print(f"Процент пользователей, которые проходили или начинали обучение несоклько раз и оплатили"
          f"  {percent_user_more_tut_and_purch}%")

Процент пользователей, которые проходили или начинали обучение несоклько раз и оплатили  13.47%


In [ ]:
#Строим сводную таблицу, чтобы найти пользователей, которые один раз начинали обучение или один раз закончили
#Найдем кол-во пользователей
pivot_start_tut = events_df_2018_users_start_tutorial.groupby(['user_id', 'event_type'])['event_type'].count().unstack()
pivot_start_tut_1_time = pivot_start_tut[(pivot_start_tut['tutorial_start'] == 1) | (pivot_start_tut['tutorial_finish'] == 1)]
users_start_tut_1_time = pivot_start_tut_1_time.index
count_users_start_tut_1_time=len(users_start_tut_1_time)
count_users_start_tut_1_time


9564

In [ ]:
#Найдем кол-во пользователей, которые прошли обучение один раз и оплатили
purchase_df_2018_pur_start_tut_1_time = purchase_df_2018[purchase_df_2018['user_id'].isin(users_start_tut_1_time)]
count_users_start_tut_1_time_and_pur = purchase_df_2018_pur_start_tut_1_time['user_id'].nunique()
count_users_start_tut_1_time_and_pur

1267

In [ ]:
#Найдем процент пользователей, которые прошли или начинали обучение один раз и оплатили
percent_user_1_time_tut_and_purch = round(count_users_start_tut_1_time_and_pur/count_users_start_tut_1_time *100,2)
percent_user_1_time_tut_and_purch
print(f"Процент пользователей, которые проходили или начинали обучение один раз и оплатили"
          f"  {percent_user_1_time_tut_and_purch}%")

Процент пользователей, которые проходили или начинали обучение один раз и оплатили  13.25%


In [ ]:
#Создание Series для построения диаграммы
result_1 = pd.Series(
    data = [count_users_start_tut_1_time_and_pur, count_users_start_tut_1_time - count_users_start_tut_1_time_and_pur],
    index = ["Процент пользователей, которые проходили или начинали обучение несколько раз и оплатили", "Процент пользователей, которые проходили или начинали обучение один раз" ]
)

result_2 = pd.Series(
    data = [count_users_start_tut_more_1_and_pur, count_users_start_tut_more_1 - count_users_start_tut_more_1_and_pur],
    index = ["Процент пользователей, которые проходили или начинали обучение несоклько раз и оплатили", "Процент пользователей, которые проходили или начинали обучение несколько раз" ]
)


In [ ]:
#Построение диграммы
fig_1 = px.pie( result_1,
    values = [count_users_start_tut_1_time_and_pur, count_users_start_tut_1_time - count_users_start_tut_1_time_and_pur],
    names = result_1.index
);
fig_1.update_layout(title_text='Небольшй разница в проценте пользователей, прошедших обучение один раз и совершивших оплату и прошедших обучение несколько раз и совершивших оплату')
fig_1.show()


fig_2 = px.pie( result_2,
    values = [count_users_start_tut_more_1_and_pur, count_users_start_tut_more_1 - count_users_start_tut_more_1_and_pur],
    names = result_2.index
);
fig_2.show()

# Вывод по результатам проверки гипотезы № 2
Процент пользователей, которые проходили или начинали обучение несколько раз и оплатили  13.47

Процент пользователей, которые проходили или начинали обучение один раз и оплатили  13.25

Проверка гипотезы № 2 показала, что частота прохождения обучения почти не влияет на вероятность оплаты пользователем.

# Проверка гипотезы № 3
Определить, насколько прозрачен процесс взаимодействия с игрой.

In [ ]:
#Фильтруем таблицу пользователей, прошедших обучение. Сортируем по start_time
mask_tut_or_level = (events_df_2018_users_start_tutorial['event_type'] == 'tutorial_start') | (events_df_2018_users_start_tutorial['event_type'] == 'level_choice') | (events_df_2018_users_start_tutorial['event_type'] == 'pack_choice')
events_df_2018_users_start_tutorial_m= events_df_2018_users_start_tutorial[mask_tut_or_level]
events_df_2018_users_start_tutorial_m = events_df_2018_users_start_tutorial_m.sort_values('start_time')


In [ ]:
#Построим пути пользователей
user_path_df = (events_df_2018_users_start_tutorial_m.groupby(["user_id"])["event_type"].apply(list).reset_index())
user_path_df["event_path"] = user_path_df["event_type"].apply(lambda x: " > ".join(x))
user_paths = (user_path_df.groupby(["event_path"])["user_id"].nunique().sort_values(ascending=False))
print(user_paths)
users_choice_level = user_paths.sum() - user_paths['tutorial_start'] - user_paths['tutorial_start > tutorial_start'] - user_paths['tutorial_start > tutorial_start > tutorial_start'] 
print(f"Общее кол-во пользователей, которые долшли до выбора уровня"
          f"  {users_choice_level}")


event_path
tutorial_start > level_choice > pack_choice                                                                                                                                            4347
tutorial_start                                                                                                                                                                         2796
tutorial_start > level_choice                                                                                                                                                          1954
tutorial_start > tutorial_start                                                                                                                                                         476
tutorial_start > level_choice > pack_choice > tutorial_start                                                                                                                            419
                                                 

In [ ]:
#Находим кол-во пользователей и процент, которые проходили обучение после выбора уровня
s = 'level_choice'
c = []
for i in user_paths.index:
  if 'level_choice > pack_choice > tutorial_start' in i or 'level_choice > tutorial_start' in i:
    c.append(i)
c_siries =  pd.Series(data = c)
c_siries
user_paths_2 = user_paths[user_paths.index.isin(c_siries)]
users_tut_after_choice_level=user_paths_2.sum()
print(f"Кол-во пользователей, которые проходили обучение после выбора уровня"
          f"  {users_tut_after_choice_level}")
percent_users_tut_after_choice_level = round(users_tut_after_choice_level/users_choice_level *100,2)
print(f"Процент пользователей, которые проходили обучение после выбора уровня"
          f"  {percent_users_tut_after_choice_level} %")

Кол-во пользователей, которые проходили обучение после выбора уровня  1386
Процент пользователей, которые проходили обучение после выбора уровня  16.4 %


In [ ]:
users_stop_on_level_choice = round(user_paths['tutorial_start > level_choice'] / users_choice_level*100,2)
print(f"Процент пользователей, которые остановились на выборе уровня"
          f"  {users_stop_on_level_choice} %")


Процент пользователей, которые остановились на выборе уровня  23.12 %


# Вывод по результатам проверки гипотезы № 3
Процент пользователей, которые проходили обучение после выбора уровня  16.15 %
Процент пользователей, которые остановились на выборе уровня  23.12 %
Проверка гипотезы № 3 показала, что достаточно высокий пароцент пользователей, которые повторно проходят обучение повсле выбора уровня. Это связано, скорее всего с непонятен дальнейший процесс, и они позвращаются к обучению. Также 23.12% пользвоателей вообще не переходят на шаг с выбором вопроса или на повторное обучение. Возможно, это также связано с неудобным или непонятным интерефейсом. 